# Quantization
This notebook acts as an example of how to use the quantization techniques.

## Setup
* Import the necessary packages.
* Load a model.
* Load a dataset.
* Analyze performance of the model prior to quantization.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import importlib
import inspect
import sys
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F

# Add thesis package to path
sys.path.append("../")
sys.path.append("../src")

import src.general as general
import src.metrics as metrics
import src.evaluation as eval
import src.compression.quantization as quant
import src.plot as plot


In [3]:
model_state = "../models/mnist.pt"
model_class = "models.mnist"

# Import the module classes
module = importlib.import_module(model_class)
classes = general.get_module_classes(module)
for cls in classes:
    globals()[cls.__name__] = cls

# Get device
device = general.get_device()

# Load the model
model = torch.load(model_state, map_location=torch.device(device))

Using cuda: False


In [4]:
# Load MNIST dataset
batch_size = 64
test_batch_size = 1000
use_cuda = False

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
mnist_transform = transforms.ToTensor()
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=mnist_transform,),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=mnist_transform,),
    batch_size=test_batch_size, shuffle=True, **kwargs)

### Pre-Quantization Evaluation

In [41]:
criterion = F.nll_loss
metric = metrics.accuracy

before_results = eval.get_results(model, test_loader, criterion, metric)
plot.print_results(**before_results)


Using cuda: False


Test: 100%|██████████| 60/60 [00:04<00:00, 14.23it/s]

================================ TEST PERFORMANCE ==============================
Average loss = 0.0767
Metric = 0.9771
Elapsed time = 4218.61 milliseconds (70.31 per batch, 0.07 per data point)
==================================== RESULTS ===================================
Loss: 0.076667
Score: 0.977067
Time per data point: 0.0703 ms
Model Size: 1.65 MB
Number of parameters: 431080
Number of FLOPs: 2307720
Number of MACs: 2307720.51


## Quantization

### Dynamic Quantization
Here the model’s weights are pre-quantized; the activations are quantized on-the-fly (“dynamic”) during inference. 

Currently only Linear and Recurrent (LSTM, GRU, RNN) layers are supported for dynamic quantization.

In [39]:
dynamic_quantized_model = quant.dynamic_quantization(model)

In [44]:
dynamic_quantized_results = eval.get_results(dynamic_quantized_model, test_loader, criterion, metric)
plot.print_before_after_results(before_results, dynamic_quantized_results)

Using cuda: False


Test: 100%|██████████| 60/60 [00:04<00:00, 13.68it/s]

================================ TEST PERFORMANCE ==============================
Average loss = 0.0766
Metric = 0.9773
Elapsed time = 4385.61 milliseconds (73.09 per batch, 0.07 per data point)
============================= RESULTS BEFORE & AFTER ===========================
Loss: 0.076667 -> 0.076579
Score: 0.977067 -> 0.977283 
Time per data point: 0.0703 ms -> 0.0731 ms
Model Size: 1.65 MB -> 0.49 MB
Number of parameters: 431080 -> 25570
Number of FLOPs: 2307720 -> 1902720
Number of MACs: 2307720.51 -> 1902720.0


### Static Quantization
Post Training Static Quantization (PTQ) also pre-quantizes model weights but instead of calibrating activations on-the-fly, the clipping range is pre-calibrated and fixed (“static”) using validation data.

In [46]:
static_quantized_model = quant.static_quantization(model, device, train_loader)

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, quant_min=0, quant_max=127){}, weight=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric){})


Test: 100%|██████████| 938/938 [00:06<00:00, 135.63it/s]

================================ TEST PERFORMANCE ==============================
Average loss = 0.0767
Elapsed time = 6917.16 milliseconds (7.37 per batch, 0.23 per data point)
Static Quantization: Calibration done



/Users/abel/miniconda3/envs/torch-gpu/lib/python3.8/site-packages/torch/ao/quantization/utils.py:280: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [47]:
static_quantized_results = eval.get_results(static_quantized_model, test_loader, criterion, metric)
plot.print_before_after_results(before_results, static_quantized_results)

Using cuda: False


Test:   0%|          | 0/60 [00:00<?, ?it/s]


NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [Metal, Negative, UNKNOWN_TENSOR_TYPE_ID, QuantizedXPU, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseCPU, SparseCUDA, SparseHIP, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseVE, UNKNOWN_TENSOR_TYPE_ID, NestedTensorCUDA, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID].

MPS: registered at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484780698/work